Resampling / Resizing

In [4]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [1]:
import nibabel as nib
import numpy as np
import os
import scipy.ndimage

# Define target shape (Height, Width, Depth)
target_shape = (256, 256, 128)

def resample_3d(image, target_shape):
    """ Resizes a 3D NIfTI image to a given shape using interpolation. """
    zoom_factors = np.array(target_shape) / np.array(image.shape)
    return scipy.ndimage.zoom(image, zoom_factors, order=3)  # order=3 is cubic interpolation

# Paths
input_folder = r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\nifti"
output_folder = r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\preprocessed"
os.makedirs(output_folder, exist_ok=True)

# Process all NIfTI files
for file in os.listdir(input_folder):
    if file.endswith(".nii.gz"):
        img_path = os.path.join(input_folder, file)
        img_nifti = nib.load(img_path)
        img_data = img_nifti.get_fdata()

        # Resample
        resized_img = resample_3d(img_data, target_shape)

        # Save the new NIfTI file
        new_img_nifti = nib.Nifti1Image(resized_img, affine=img_nifti.affine)
        nib.save(new_img_nifti, os.path.join(output_folder, file))
        
        print(f" Resampled & Saved: {file}")


In [1]:
import os
import nibabel as nib
import numpy as np
from PIL import Image

class MRIDataset:
    def __init__(self, root, mode="train", transform=None, load_segmentation=True):
        self.root = root
        self.mode = mode
        self.dataset_path = os.path.join(self.root, self.mode)
        self.transform = transform
        self.load_segmentation = load_segmentation

        self.image_paths, self.mask_paths = self.load_image_paths()

    def load_image_paths(self):
        """Loads .nii.gz files from cet1 and pret1 folders and converts them to NumPy arrays."""
        image_paths = []
        mask_paths = {}

        for patient_folder in os.listdir(self.dataset_path):
            patient_path = os.path.join(self.dataset_path, patient_folder)
            if not os.path.isdir(patient_path):
                continue

            print(f" Found patient folder: {patient_folder}")

            for subfolder in ["cet1", "pret1"]:
                subfolder_path = os.path.join(patient_path, subfolder)
                if os.path.exists(subfolder_path):
                    print(f"🔍 Checking {subfolder_path}")

                    for file in os.listdir(subfolder_path):
                        if file.endswith('.nii.gz'):
                            nii_path = os.path.join(subfolder_path, file)
                            image_paths.append(nii_path)
                            print(f" Found NIfTI image: {nii_path}")

            # Check segmentation masks
            segmentation_path = os.path.join(patient_path, "segmentation")
            if os.path.exists(segmentation_path):
                mask_files = [os.path.join(segmentation_path, f) for f in os.listdir(segmentation_path)
                              if f.endswith('.nii.gz')]
                mask_paths[patient_folder] = sorted(mask_files)
                print(f" Found {len(mask_files)} segmentation masks in: {segmentation_path}")

        if not image_paths:
            raise FileNotFoundError(f" No .nii.gz images found in {self.dataset_path}")

        return sorted(image_paths), mask_paths

    def __getitem__(self, idx):
        """Loads an MRI scan and its corresponding segmentation mask (if available)."""
        image_path = self.image_paths[idx]

        # Load NIfTI image
        nii_image = nib.load(image_path)
        image_data = nii_image.get_fdata()  # Convert to NumPy array
        image_data = np.array(image_data, dtype=np.float32)

        # Normalize the image (0-1 range)
        image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))

        # Convert to grayscale image (selecting middle slice)
        mid_slice = image_data.shape[2] // 2
        image_slice = image_data[:, :, mid_slice]

        # Convert NumPy array to PIL Image for visualization
        pil_image = Image.fromarray((image_slice * 255).astype(np.uint8))

        return pil_image

    def __len__(self):
        return len(self.image_paths)


#  Test the dataset loader
dataset = MRIDataset(root="C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset", mode="train")

#  Display a sample image
sample_image = dataset[0]
sample_image.show()  # Opens the image in the default viewer

 Found patient folder: ISPY1_1001
🔍 Checking C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\cet1
 Found NIfTI image: C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\cet1\31000_dynamic-3dfgre_ser.nii.gz
 Found NIfTI image: C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\cet1\3_dynamic-3dfgre.nii.gz
 Found NIfTI image: C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\cet1\41001_dynamic-3dfgre_pe1.nii.gz
🔍 Checking C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\pret1
 Found NIfTI image: C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\pret1\1_t1-axial-locator.nii.gz
 Found NIfTI image: C:/Users/OMEN/Desktop/MRI_Synthesis_Project/MRI_SYNTHESIS_PROJECT/data/dataset\train\ISPY1_1001\pret1\2_t1-axial-locator.n

In [ ]:
"""
Data Check Script for MRI Synthesis Project

Run this before training to verify your data is properly organized.
"""
import os
import sys
import glob
import nibabel as nib
import numpy as np

def check_data_structure(base_dir):
    """Check the patient-level data directory structure and files."""
    print(f"\n{'='*50}")
    print(f"Checking data directory: {base_dir}")
    print(f"{'='*50}")
    
    # 1. Check if base directory exists
    if not os.path.exists(base_dir):
        print(f"❌ Base directory doesn't exist: {base_dir}")
        return False
    
    print(f"✅ Base directory exists")
    
    # 2. List patient directories
    patient_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    
    if not patient_dirs:
        print("❌ No patient directories found")
        return False
    
    print(f" Found {len(patient_dirs)} patient directories")
    print(f"Sample patients: {patient_dirs[:5]}")
    
    # 3. Define the expected subdirectories in each patient folder
    required_dirs = {
        'pre': ['PreT1', 'pret1', 'pre_t1', 'pre-t1', 'preT1', 'Pre_T1', 'Pre'],
        'ce': ['CeT1', 'cet1', 'ce_t1', 'ce-t1', 'ceT1', 'Ce_T1', 'Ce'],
        'seg': ['Seg', 'seg', 'segmentation', 'Segmentation', 'masks', 'Masks']
    }
    
    # 4. Count valid patients and collect file paths
    valid_patients = 0
    pre_paths = []
    ce_paths = []
    seg_paths = []
    extensions = ['*.nii.gz', '*.nii', '*.img']
    
    for patient in patient_dirs:
        patient_path = os.path.join(base_dir, patient)
        print(f"\n{'-'*30}")
        print(f"Checking patient: {patient}")
        
        # Get available subdirectories for this patient
        patient_subdirs = os.listdir(patient_path)
        print(f"  Subdirectories: {patient_subdirs}")
        
        # Find matching directories for each image type
        found_dirs = {}
        for data_type, possible_names in required_dirs.items():
            found = next((d for d in possible_names if d in patient_subdirs), None)
            found_dirs[data_type] = found
            
            if found:
                print(f"  ✅ Found {data_type} directory: {found}")
            else:
                print(f"  ❌ Missing {data_type} directory")
        
        # Skip patients with missing directories
        if not all(found_dirs.values()):
            print(f"  ⚠️ Skipping patient {patient} - missing required directories")
            continue
        
        # Check for files with valid extensions
        patient_files = {}
        valid_extension = None
        
        for ext in extensions:
            files = {}
            for data_type, dir_name in found_dirs.items():
                pattern = os.path.join(patient_path, dir_name, ext)
                matching_files = glob.glob(pattern)
                files[data_type] = matching_files
            
            # Check if we found files for all types
            if all(files.values()):
                valid_extension = ext
                patient_files = files
                print(f"  ✅ Found files with extension: {ext}")
                break
            else:
                present = {k: len(v) > 0 for k, v in files.items()}
                print(f"  Extension {ext}: {present}")
        
        if not valid_extension:
            print(f"  ⚠️ Skipping patient {patient} - no valid files found")
            continue
        
        # Add patient's files to our collections
        pre_paths.append(patient_files['pre'][0])  # Take first file if multiple
        ce_paths.append(patient_files['ce'][0])
        seg_paths.append(patient_files['seg'][0])
        valid_patients += 1
        
        # Check sample file from this patient
        try:
            sample_file = patient_files['pre'][0]
            img = nib.load(sample_file)
            data = img.get_fdata()
            
            print(f"  ✅ Sample file loads successfully: {os.path.basename(sample_file)}")
            print(f"    - Shape: {data.shape}")
            print(f"    - Range: {data.min():.2f} to {data.max():.2f}")
            
        except Exception as e:
            print(f"  ❌ Error loading file: {str(e)}")
    
    print(f"\n{'='*50}")
    print(f"Summary")
    print(f"{'='*50}")
    print(f"Total patients: {len(patient_dirs)}")
    print(f"Valid patients with all data: {valid_patients}")
    
    if valid_patients == 0:
        print(f"❌ No valid patients found with complete data")
        return False
    
    print(f"✅ Found {valid_patients} valid patients with all required data")
    
    # Check a sample file more thoroughly
    if pre_paths:
        try:
            sample_file = pre_paths[0]
            print(f"\nDetailed check of sample file: {os.path.basename(sample_file)}")
            
            img = nib.load(sample_file)
            data = img.get_fdata()
            
            print(f"  - Shape: {data.shape}")
            print(f"  - Data type: {data.dtype}")
            print(f"  - Data range: {data.min():.2f} to {data.max():.2f}")
            print(f"  - Header info: {img.header.get_data_shape()}")
            
            # Basic validation
            if data.ndim != 3:
                print(f"⚠️ WARNING: Expected 3D data, got {data.ndim}D")
            
            if np.isnan(data).any():
                print("⚠️ WARNING: Data contains NaN values")
                
        except Exception as e:
            print(f"❌ Error in detailed file check: {str(e)}")
    
    return True

if __name__ == "__main__":
    # Use the provided directory or default
    data_dir = sys.argv[1] if len(sys.argv) > 1 else r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\preprocessed"
    
    if check_data_structure(data_dir):
        print("\n✅ Data structure looks good! You should be able to run training.")
    else:
        print("\n❌ Data structure has issues. Please fix before running training.")


Checking data directory: --f=c:\Users\OMEN\AppData\Roaming\jupyter\runtime\kernel-v3f7904480181d4830a27fd322c457de30a7e9ec4d.json
❌ Base directory doesn't exist: --f=c:\Users\OMEN\AppData\Roaming\jupyter\runtime\kernel-v3f7904480181d4830a27fd322c457de30a7e9ec4d.json

❌ Data structure has issues. Please fix before running training.
